# House Extractor

In [1]:
from bs4 import BeautifulSoup
import requests
import csv
import os
import pandas as pd

In [17]:
# to be clean creating a python "switch" statement:

def switch_unlabeledFields(args):
    switcher = {1: 'Address:',
                2: 'Room:',
                3: 'City:',
                4: 'Province:',
                5: 'Post:',
                9: 'Street:',
                10: 'District:',
                14: 'Year:'
                16: 'house_type:',
                17: 'house_style:'}
    return switcher.get(args, str(args) + ':')

In [22]:
house_detail_dictionary = {}
# 
# header_indicator = True

with open('house_data.csv', 'a') as csv_file:    
    for filename in os.listdir(os.getcwd()):
        if filename.endswith('.html'):
            with open(filename) as html_file:
                soup = BeautifulSoup(html_file, 'lxml')
                date_printed = soup.find('div', class_='header').text
                date_printed = (date_printed[date_printed.find('Printed') + 11:date_printed.find('PM')-10])
                source = filename
                div_house_list = [tag for tag in soup.find_all('div', class_='report-container') if len(tag.get('class')) > 1]
                #report-container has multiple classes so have to specify only pick those with multiple classes
                for div_house in div_house_list:
                    house_spans = div_house.find_all('span', class_='formitem formfield')
                    address = div_house.find('div')
                    i = 0
                    for span in house_spans:
                        i = i + 1
                        label = span.label
                        detail = span.find('span', class_='value')
                        if label is not None:
                            print(label.text + ' ' + detail.text)
                            house_detail_dictionary[label.text[:-1]] = detail.text
                        if label is None and i <= 30:
                            label = switch_unlabeledFields(i)                
                            print (label + ' ' + detail.text)
                            house_detail_dictionary[label[:-1]] = detail.text
                    print ('date printed: ' + date_printed)
                    print ('source:' + source)
                    house_detail_dictionary['DateAdded'] = date_printed
                    house_detail_dictionary['Source'] = source
                    w = csv.DictWriter(csv_file, house_detail_dictionary.keys(), lineterminator='\n')
#                     if header_indicator == True:
#                         w.writeheader()
#                         header_indicator = False
                    w.writerow(house_detail_dictionary)
                    print ()

Address: 55 Bristol Rd E
Room: 112
City: Mississauga
Province: Ontario
Post: L4Z3N9
List: $378,000
For: Sale
8: Mississauga
Street: Hurontario
District Peel
11: 466-42-G
SPIS: N
Taxes: $1,927.00
14: 2018
Last Status: Sc
house_type: Condo Apt
house_style: Stacked Townhse
Corp#: PCC / 479
#Shares%: 
Locker#: 
Locker Level: 
Locker Unit#: 
Level: 3
Unit#: 6
Rms: 5
Bedrooms: 2
Washrooms: 1
28: 1x4
Dir/Cross St: Hwy 10 & Bristol
Zoning: 
MLS#: W4235318
Possession: 11/02/2018
PIN#: 
Kitchens: 1
Fam Rm: N
Basement: None
Fireplace/Stv: N
Heat: Forced Air / Gas
Apx Age: 
Apx Sqft: 900-999
Sqft Source: As Per Seller
Exposure: N
Assessment: 250000 / 2018
Spec Desig: Unknown
Phys Hdp-Eqp: 
Pets Perm: Restrict
Locker: Exclusive
Maint: $479.00
A/C: Central Air
Central Vac: 
UFFI: 
Elev/Lift: 
Retirement: 
Taxes Incl: N
Water Incl: Y
Heat Incl: N
Hydro Incl: N
Cable TV Incl: N
CAC Incl: N
Bldg Ins Incl: Y
Prkg Incl: Y
Cert Level: 
Energy Cert: 
GreenPIS: 
Balcony: Open
Ens Lndry: Y
Lndy Lev: Main
Ext

Address: 4450 Tucana Crt
Room: 308
City: Mississauga
Province: Ontario
Post: L5R 3R4
List: $389,000
For: Sale
8: Mississauga
Street: Hurontario
District Peel
11: 466-42-J
SPIS: N
Taxes: $1,865.17
14: 2018
Last Status: Sc
house_type: Condo Apt
house_style: Apartment
Corp#: PCC / 455
#Shares%: 
Locker#: 
Locker Level: 
Locker Unit#: 
Level: 3
Unit#: 308
Rms: 6
Bedrooms: 2
Washrooms: 2
28: 2x4
Dir/Cross St: Hurontario /Kingsbridge Garden
Zoning: 
MLS#: W4221227
Possession: Flexible
PIN#: 
Kitchens: 1
Fam Rm: N
Basement: None
Fireplace/Stv: N
Heat: Forced Air / Gas
Apx Age: 
Apx Sqft: 1200-1399
Sqft Source: Mpac
Exposure: Ne
Assessment: 
Spec Desig: Unknown
Phys Hdp-Eqp: 
Pets Perm: Restrict
Locker: Exclusive
Maint: $933.88
A/C: Central Air
Central Vac: N
UFFI: 
Elev/Lift: 
Retirement: 
Taxes Incl: N
Water Incl: Y
Heat Incl: Y
Hydro Incl: N
Cable TV Incl: N
CAC Incl: Y
Bldg Ins Incl: Y
Prkg Incl: Y
Cert Level: 
Energy Cert: 
GreenPIS: 
Balcony: None
Ens Lndry: Y
Lndy Lev: Main
Exterior: Co

Address: 50 Kingsbridge Garden Circ
Room: 407
City: Mississauga
Province: Ontario
Post: L5R1Y2
List: $415,000
For: Sale
8: Mississauga
Street: Hurontario
District Peel
11: 466-42-J
SPIS: N
Taxes: $1,788.00
14: 2018
Last Status: Sc
house_type: Condo Apt
house_style: Apartment
Corp#: PCC / 338
#Shares%: 
Locker#: 
Locker Level: B
Locker Unit#: 31
Level: 4
Unit#: 407
Rms: 5
Bedrooms: 2 + 1
Washrooms: 1
28: 1x4xMain
Dir/Cross St: Highway 10/Eglinton
Zoning: 
MLS#: W4236105
Possession: Tbd
PIN#: 
Kitchens: 1
Fam Rm: N
Basement: None
Fireplace/Stv: N
Heat: Forced Air / Gas
Apx Age: 
Apx Sqft: 800-899
Sqft Source: Mls
Exposure: Se
Assessment: 
Spec Desig: Unknown
Phys Hdp-Eqp: 
Pets Perm: Restrict
Locker: Owned
Maint: $673.33
A/C: Central Air
Central Vac: 
UFFI: 
Elev/Lift: Y
Retirement: 
Taxes Incl: N
Water Incl: Y
Heat Incl: Y
Hydro Incl: Y
Cable TV Incl: N
CAC Incl: Y
Bldg Ins Incl: Y
Prkg Incl: Y
Cert Level: 
Energy Cert: 
GreenPIS: 
Balcony: None
Ens Lndry: Y
Lndy Lev: Main
Exterior: Con

Water Incl: Y
Heat Incl: N
Hydro Incl: N
Cable TV Incl: N
CAC Incl: N
Bldg Ins Incl: Y
Prkg Incl: Y
Cert Level: 
Energy Cert: N
GreenPIS: 
Balcony: None
Ens Lndry: Y
Lndy Lev: Lower
Exterior: Brick
Gar/Gar Spcs: Attached / 1.0
Park/Drive: Mutual
Park Type: Exclusive
Park/Drv Spcs: 1
Tot Pk Spcs: 2.0
Park $/Mo: 
Prk Lvl/Unit: 
Com Elem Incl: Y
Bldg Amen: Visitor Parking
Prop Feat: Golf, Place Of Worship, Public Transit, School, School Bus Route
Client Remks: Welcome To 32-5255 Guildwood Way. A Rare 3 Bdrm Unit In A High Demand Complex. This Home Is Located In The Mavis And Eglinton Area, Near The Heart Of Mississauga. Close To Parks, Heartland Shopping Ctre, Square 1 Mall, Resto's, Groceries, School, Places Of Worship, Major H-Way 401/403 & Go Train Stn. Well  Maintained, Lightly Lived In + Lower Lev. Has Some Wiring Done & Has Rough-In 3Pce Bath, Hi Ceil & Reno Ready Furance & Ac Rental $151.25/Mo W/Ecohome.
Extras: Included: Fridge, Stove, Above Range Microwave, Washer & Dryer, All El

Address: 103 Bristol Rd E
Room: 202
City: Mississauga
Province: Ontario
Post: L4Z3P6
List: $429,900
For: Sale
8: Mississauga
Street: Hurontario
District Peel
11: 466-42-G
SPIS: N
Taxes: $2,066.92
14: 2018
Last Status: New
house_type: Condo Apt
house_style: Stacked Townhse
Corp#: PCC / 469
#Shares%: 
Locker#: 
Locker Level: 
Locker Unit#: 
Level: 2
Unit#: 56
Rms: 6
Bedrooms: 3
Washrooms: 1
28: 1x4
Dir/Cross St: Hurontario/Bristol
Zoning: 
MLS#: W4249722
Possession: 30-60-90
PIN#: 
Kitchens: 1
Fam Rm: N
Basement: None
Fireplace/Stv: N
Heat: Forced Air / Gas
Apx Age: 
Apx Sqft: 1200-1399
Sqft Source: Owner
Exposure: S
Assessment: 
Spec Desig: Unknown
Phys Hdp-Eqp: 
Pets Perm: Restrict
Locker: Ensuite
Maint: $431.57
A/C: Central Air
Central Vac: N
UFFI: No
Elev/Lift: 
Retirement: 
Taxes Incl: N
Water Incl: Y
Heat Incl: N
Hydro Incl: N
Cable TV Incl: N
CAC Incl: N
Bldg Ins Incl: Y
Prkg Incl: Y
Cert Level: 
Energy Cert: 
GreenPIS: 
Balcony: Open
Ens Lndry: Y
Lndy Lev: Main
Exterior: Brick
Ga

...

Address: 29 Sandringham Crt
Room: 
City: Brampton
Province: Ontario
Post: L6T 3Z3
List: $452,900
For: Sale
8: Brampton
Street: Bram West
District Peel
11: 
SPIS: Y
Taxes: $2,513.85
14: 2018
Last Status: New
house_type: Condo Townhouse
house_style: 2-Storey
Corp#: PCC / 150
#Shares%: 
Locker#: 
Locker Level: 
Locker Unit#: 
Level: 1
Unit#: 29
Rms: 6 + 1
Bedrooms: 3
Washrooms: 2
28: 1x4xUpper, 1x2xMain
Dir/Cross St: Dixie/Hwy 7
Zoning: 
MLS#: W4255705
Possession: 60 Days Tba
PIN#: 
Kitchens: 1
Fam Rm: N
Basement: Fin W/O
Fireplace/Stv: N
Heat: Forced Air / Gas
Apx Age: 
Apx Sqft: 1200-1399
Sqft Source: Listing Agent
Exposure: N
Assessment: 
Spec Desig: Unknown
Phys Hdp-Eqp: 
Pets Perm: Restrict
Locker: None
Maint: $367.11
A/C: Central Air
Central Vac: 
UFFI: 
Elev/Lift: 
Retirement: 
Taxes Incl: N
Water Incl: Y
Heat Incl: N
Hydro Incl: N
Cable TV Incl: Y
CAC Incl: N
Bldg Ins Incl: Y
Prkg Incl: Y
Cert Level: 
Energy Cert: 
GreenPIS: 
Balcony: None
Ens Lndry: N
Lndy Lev: 
Exterior: Brick /

District Peel
11: 453-50-V
SPIS: N
Taxes: $2,847.88
14: 2018
Last Status: New
house_type: Condo Townhouse
house_style: 2-Storey
Corp#: PCC / 68
#Shares%: 
Locker#: 
Locker Level: 
Locker Unit#: 
Level: 1
Unit#: 82
Rms: 7
Bedrooms: 4
Washrooms: 3
28: 2x4x2nd, 0x0, 1x2xMain
Dir/Cross St: Queen/Glenvale
Zoning: 
MLS#: W4255250
Possession: Tba
PIN#: 
Kitchens: 1
Fam Rm: N
Basement: Finished
Fireplace/Stv: Y
Heat: Forced Air / Gas
Apx Age: 16-30
Apx Sqft: 1400-1599
Sqft Source: As Per Seller
Exposure: E
Assessment: 
Spec Desig: Unknown
Phys Hdp-Eqp: 
Pets Perm: Restrict
Locker: None
Maint: $444.70
A/C: Central Air
Central Vac: 
UFFI: No
Elev/Lift: 
Retirement: 
Taxes Incl: N
Water Incl: Y
Heat Incl: N
Hydro Incl: N
Cable TV Incl: Y
CAC Incl: Y
Bldg Ins Incl: Y
Prkg Incl: Y
Cert Level: 
Energy Cert: 
GreenPIS: 
Balcony: None
Ens Lndry: Y
Lndy Lev: Lower
Exterior: Brick / Alum Siding
Gar/Gar Spcs: Attached / 1.0
Park/Drive: Private
Park Type: Exclusive
Park/Drv Spcs: 1
Tot Pk Spcs: 2.0
Park $

Address: 1624 Bloor St
Room: 10
City: Mississauga
Province: Ontario
Post: L4X2S2
List: $349,900
For: Sale
8: Mississauga
Street: Applewood
District Peel
11: 473-46-L
SPIS: N
Taxes: $1,650.00
14: 2018
Last Status: New
house_type: Condo Townhouse
house_style: Stacked Townhse
Corp#: PCC / 35
#Shares%: 100
Locker#: 
Locker Level: 
Locker Unit#: 
Level: 1
Unit#: 19
Rms: 7
Bedrooms: 4
Washrooms: 3
28: 1x2xMain, 1x4x2nd, 1x1x2nd
Dir/Cross St: Dixie & Bloor
Zoning: 
MLS#: W4255860
Possession: 60/90 Tba
PIN#: 
Kitchens: 1
Fam Rm: N
Basement: None
Fireplace/Stv: N
Heat: Baseboard / Electric
Apx Age: 
Apx Sqft: 1400-1599
Sqft Source: Per Seller
Exposure: W
Assessment: 
Spec Desig: Unknown
Phys Hdp-Eqp: 
Pets Perm: Restrict
Locker: Exclusive
Maint: $824.45
A/C: Window Unit
Central Vac: 
UFFI: 
Elev/Lift: 
Retirement: 
Taxes Incl: N
Water Incl: Y
Heat Incl: Y
Hydro Incl: N
Cable TV Incl: Y
CAC Incl: N
Bldg Ins Incl: Y
Prkg Incl: Y
Cert Level: 
Energy Cert: N
GreenPIS: 
Balcony: Open
Ens Lndry: N
Ln

Assessment: 
Spec Desig: Unknown
Phys Hdp-Eqp: 
Pets Perm: Restrict
Locker: None
Maint: $395.00
A/C: Central Air
Central Vac: 
UFFI: 
Elev/Lift: 
Retirement: 
Taxes Incl: N
Water Incl: N
Heat Incl: N
Hydro Incl: N
Cable TV Incl: N
CAC Incl: N
Bldg Ins Incl: Y
Prkg Incl: Y
Cert Level: 
Energy Cert: 
GreenPIS: 
Balcony: Open
Ens Lndry: Y
Lndy Lev: Lower
Exterior: Brick
Gar/Gar Spcs: Attached / 1.0
Park/Drive: Private
Park Type: Exclusive
Park/Drv Spcs: 2
Tot Pk Spcs: 3.0
Park $/Mo: 
Prk Lvl/Unit: 
Com Elem Incl: Y
Bldg Amen: 
Prop Feat: Cul De Sac, Park, Place Of Worship, Public Transit, Rec Centre, School
Client Remks: Modern 3Br+1, 3Wr Condo Townhouse With W/O Bsmt In The East Credit Area Minutes Away From Heartland Town Center! Spacious Layout W/ Quality Finishes & Upgrade T/O Incld: Spiral Staircase, Hardwood Floors, California Shutters, Pot Lights. Open Concept Living Rm W/ Gas Fireplace W/ W/O To Balcony. Modern Kitchen W/ B/I Appliances & Ceramic Flr W/ Breakfast Area O/L Backyard

Heat Incl: N
Hydro Incl: N
Cable TV Incl: N
CAC Incl: Y
Bldg Ins Incl: Y
Prkg Incl: Y
Cert Level: 
Energy Cert: 
GreenPIS: 
Balcony: None
Ens Lndry: Y
Lndy Lev: Lower
Exterior: Brick
Gar/Gar Spcs: Attached / 1.0
Park/Drive: Private
Park Type: Owned
Park/Drv Spcs: 1
Tot Pk Spcs: 2.0
Park $/Mo: 
Prk Lvl/Unit: 
Com Elem Incl: Y
Bldg Amen: 
Prop Feat: Hospital, Park, Public Transit
Client Remks: Fully Renovated By Daniels Builder In 2014, Built In 2001, New Paint In Entire Townhouse,2 Storey Townhouse With Finished Walkout Basmt. With Built In Garage (2) Two Parking Space, Close To Erin Mills Town Centre Mall, Credit Valley Hospital, Walmart, John Fraser Secondary Shcool, Loblaws Store, Minutes To Hwy 403, Close To Main Transportation, Children Play Area, Library, Community Centre, Bus Stop, Quite Neighbourhood Available For Immediate Possession.
Extras: Fridge, Stove, Built In Dishwasher, Finished Walk Out Basement, Lots Of Space For Storage, Window Coverings And All Elf's. Please See The